## 导包

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import torch
import aiohttp
import asyncio
import json
import requests
from load_test import run_load_test

## 数据处理

<font color=red>数据已处理好，无需再次运行</font>

In [2]:
!python netflix_data_convert.py Netflix_data/ratings Netflix_data/output

Processing: Netflix_data/ratings\mv_0000004.txt
Processing: Netflix_data/ratings\mv_0000008.txt
Processing: Netflix_data/ratings\mv_0000009.txt
Processing: Netflix_data/ratings\mv_0000014.txt
Processing: Netflix_data/ratings\mv_0000061.txt
Processing: Netflix_data/ratings\mv_0000066.txt
Processing: Netflix_data/ratings\mv_0000074.txt
Processing: Netflix_data/ratings\mv_0000088.txt
Processing: Netflix_data/ratings\mv_0000090.txt
Processing: Netflix_data/ratings\mv_0000095.txt
Processing: Netflix_data/ratings\mv_0000102.txt
Processing: Netflix_data/ratings\mv_0000152.txt
Processing: Netflix_data/ratings\mv_0000174.txt
Processing: Netflix_data/ratings\mv_0000187.txt
Processing: Netflix_data/ratings\mv_0000191.txt
Processing: Netflix_data/ratings\mv_0000204.txt
Processing: Netflix_data/ratings\mv_0000226.txt
Processing: Netflix_data/ratings\mv_0000227.txt
Processing: Netflix_data/ratings\mv_0000278.txt
Processing: Netflix_data/ratings\mv_0000299.txt
Processing: Netflix_data/ratings\mv_0000

STATS FOR ALL INPUT DATA
STATS
Total Ratings: 5974565
Total User count: 436746
Netflix full train
STATS
Total Ratings: 5835300
Total User count: 430576
Netflix full valid
STATS
Total Ratings: 57277
Total User count: 28871
Netflix full test
STATS
Total Ratings: 57538
Total User count: 28958
Netflix 3m train
STATS
Total Ratings: 786865
Total User count: 173559
Netflix 3m valid
STATS
Total Ratings: 45483
Total User count: 22190
Netflix 3m test
STATS
Total Ratings: 45611
Total User count: 22141
Netflix 6m train
STATS
Total Ratings: 1691351
Total User count: 280741
Netflix 6m valid
STATS
Total Ratings: 52781
Total User count: 26348
Netflix 6m test
STATS
Total Ratings: 52880
Total User count: 26295
Netflix 1y train
STATS
Total Ratings: 2406508
Total User count: 271348
Netflix 1y valid
STATS
Total Ratings: 90332
Total User count: 44875
Netflix 1y test
STATS
Total Ratings: 90613
Total User count: 45288


<font color=red>上面命令行中的ratings文件夹现在为压缩包形式，使用前需先进行解压</font>

In [2]:
TRAIN = 'Netflix_data/output/N3M_TRAIN'
EVAL = 'Netflix_data/output/N3M_VALID'
TEST = 'Netflix_data/output/N3M_TEST'

In [3]:
nf_3m_test = os.path.join(TEST,'n3m.test.txt')
df = pd.read_csv(nf_3m_test, names=['CustomerID','MovieID','Rating'],dtype={'MovieID':np.object}, sep='\t')
df.head()

,CustomerID,MovieID,Rating
0,6,1062,3.0
1,8,399,5.0
2,8,579,3.0
3,12,729,2.0
4,13,631,4.0


## 模型训练与评估

### 1. 设置参数

In [4]:
# Autoencoder parameters
GPUS = 0 #'0,1,2,3'
ACTIVATION = 'selu'
OPTIMIZER = 'momentum'
HIDDEN = '512,512,1024'
BATCH_SIZE = 128
DROPOUT = 0.8
LR = 0.005
WD = 0
EPOCHS = 10
AUG_STEP = 1
MODEL_OUTPUT_DIR = 'model_save'

# Evaluation
INFER_OUTPUT = 'preds.txt'
MODEL_PATH = os.path.join(MODEL_OUTPUT_DIR, 'model.epoch_' + str(EPOCHS-1))
MOVIE_TITLES = 'Netflix_data/movie_titles.txt'

### 2. 开始训练

<font color=red>模型已训练好，无需再次运行</font>

In [7]:
%run run.py --gpu_ids $GPUS \
    --path_to_train_data $TRAIN \
    --path_to_eval_data $EVAL \
    --hidden_layers $HIDDEN \
    --non_linearity_type $ACTIVATION \
    --batch_size $BATCH_SIZE \
    --logdir $MODEL_OUTPUT_DIR \
    --drop_prob $DROPOUT \
    --optimizer $OPTIMIZER \
    --lr $LR \
    --weight_decay $WD \
    --aug_step $AUG_STEP \
    --num_epochs $EPOCHS 

D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:472: FutureWarning: Passi

Namespace(aug_step=1, batch_size=128, constrained=False, drop_prob=0.8, gpu_ids='0', hidden_layers='512,512,1024', logdir='model_save', lr=0.005, noise_prob=0.0, non_linearity_type='selu', num_epochs=10, optimizer='momentum', path_to_eval_data='Netflix_data/output/N3M_VALID', path_to_train_data='Netflix_data/output/N3M_TRAIN', skip_last_layer_nl=False, summary_frequency=100, weight_decay=0.0)
GPU is not available.
Loading training data
Data loaded
Total items found: 173559
Vector dim: 1106
Loading eval data


E:\工作\课程\特训营\电商推荐系统\recommender_system\Online\lesson3\PyTorch\reco_encoder\model\model.py:60: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  weight_init.xavier_uniform(w)
E:\工作\课程\特训营\电商推荐系统\recommender_system\Online\lesson3\PyTorch\reco_encoder\model\model.py:72: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  weight_init.xavier_uniform(w)


******************************
******************************
[1106, 512, 512, 1024]
Dropout drop probability: 0.8
Encoder pass:
torch.Size([512, 1106])
torch.Size([512])
torch.Size([512, 512])
torch.Size([512])
torch.Size([1024, 512])
torch.Size([1024])
Decoder pass:
torch.Size([512, 1024])
torch.Size([512])
torch.Size([512, 512])
torch.Size([512])
torch.Size([1106, 512])
torch.Size([1106])
******************************
******************************
######################################################
######################################################
############# AutoEncoder Model: #####################
AutoEncoder(
  (drop): Dropout(p=0.8)
  (encode_w): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 512x1106]
      (1): Parameter containing: [torch.FloatTensor of size 512x512]
      (2): Parameter containing: [torch.FloatTensor of size 1024x512]
    
  )
  (encode_b): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 512]

E:\工作\课程\特训营\电商推荐系统\recommender_system\Online\lesson3\PyTorch\run.py:196: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  t_loss += loss.data[0]


[0,     0] RMSE: 3.9898903


E:\工作\课程\特训营\电商推荐系统\recommender_system\Online\lesson3\PyTorch\run.py:210: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  total_epoch_loss += loss.data[0]


[0,   100] RMSE: 1.7303204
[0,   200] RMSE: 1.1430737
[0,   300] RMSE: 1.0526411
[0,   400] RMSE: 1.0050501
[0,   500] RMSE: 0.9690841
[0,   600] RMSE: 0.9697021
[0,   700] RMSE: 0.9311695
[0,   800] RMSE: 0.9246207
[0,   900] RMSE: 0.9217271
[0,  1000] RMSE: 0.9107654
[0,  1100] RMSE: 0.9029777
[0,  1200] RMSE: 0.8939253
[0,  1300] RMSE: 0.8775005
Total epoch 0 finished in 228.30405855178833 seconds with TRAINING RMSE loss: 1.0395551388471373


E:\工作\课程\特训营\电商推荐系统\recommender_system\Online\lesson3\PyTorch\run.py:72: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  total_epoch_loss += loss.data[0]
E:\工作\课程\特训营\电商推荐系统\recommender_system\Online\lesson3\PyTorch\run.py:73: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  denom += num_ratings.data[0]


Epoch 0 EVALUATION LOSS: 1.8925109788382188
Saving model to model_save/model.epoch_0
Doing epoch 1 of 10
[1,     0] RMSE: 0.8598430
[1,   100] RMSE: 0.8506381
[1,   200] RMSE: 0.8484050
[1,   300] RMSE: 0.8413375
[1,   400] RMSE: 0.8481489
[1,   500] RMSE: 0.8276425
[1,   600] RMSE: 0.8291753
[1,   700] RMSE: 0.8208077
[1,   800] RMSE: 0.8240229
[1,   900] RMSE: 0.8338989
[1,  1000] RMSE: 0.8101237
[1,  1100] RMSE: 0.8132584
[1,  1200] RMSE: 0.8153033
[1,  1300] RMSE: 0.8072152
Total epoch 1 finished in 226.33594584465027 seconds with TRAINING RMSE loss: 0.8282605546093988
Doing epoch 2 of 10
[2,     0] RMSE: 0.8207490
[2,   100] RMSE: 0.7933994
[2,   200] RMSE: 0.7851581
[2,   300] RMSE: 0.7882614
[2,   400] RMSE: 0.7932630
[2,   500] RMSE: 0.7945206
[2,   600] RMSE: 0.7790330
[2,   700] RMSE: 0.7764686
[2,   800] RMSE: 0.7807144
[2,   900] RMSE: 0.7759949
[2,  1000] RMSE: 0.7735806
[2,  1100] RMSE: 0.7768570
[2,  1200] RMSE: 0.7805676
[2,  1300] RMSE: 0.7710443
Total epoch 2 finished

### 3. 评估

接下来我们要在测试集上进行评估<font color=red>（无需再次运行）</font>

In [8]:
%run infer.py \
--path_to_train_data $TRAIN \
--path_to_eval_data $TEST \
--hidden_layers $HIDDEN \
--non_linearity_type $ACTIVATION \
--save_path  $MODEL_PATH \
--drop_prob $DROPOUT \
--predictions_path $INFER_OUTPUT

Namespace(constrained=False, drop_prob=0.8, hidden_layers='512,512,1024', non_linearity_type='selu', path_to_eval_data='Netflix_data/output/N3M_TEST', path_to_train_data='Netflix_data/output/N3M_TRAIN', predictions_path='preds.txt', save_path='model_save\\model.epoch_9', skip_last_layer_nl=False)
GPU is not available.
Loading training data
Data loaded
Total items found: 173559
Vector dim: 1106
Loading eval data
******************************
******************************
[1106, 512, 512, 1024]
Dropout drop probability: 0.8
Encoder pass:


E:\工作\课程\特训营\电商推荐系统\recommender_system\Online\lesson3\PyTorch\reco_encoder\model\model.py:60: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  weight_init.xavier_uniform(w)
E:\工作\课程\特训营\电商推荐系统\recommender_system\Online\lesson3\PyTorch\reco_encoder\model\model.py:72: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  weight_init.xavier_uniform(w)


torch.Size([512, 1106])
torch.Size([512])
torch.Size([512, 512])
torch.Size([512])
torch.Size([1024, 512])
torch.Size([1024])
Decoder pass:
torch.Size([512, 1024])
torch.Size([512])
torch.Size([512, 512])
torch.Size([512])
torch.Size([1106, 512])
torch.Size([1106])
******************************
******************************
Loading model from: model_save\model.epoch_9
######################################################
######################################################
############# AutoEncoder Model: #####################
AutoEncoder(
  (drop): Dropout(p=0.8)
  (encode_w): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 512x1106]
      (1): Parameter containing: [torch.FloatTensor of size 512x512]
      (2): Parameter containing: [torch.FloatTensor of size 1024x512]
    
  )
  (encode_b): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 512]
      (1): Parameter containing: [torch.FloatTensor of size 512]
      (2): Paramet

In [9]:
%run compute_RMSE.py --path_to_predictions=$INFER_OUTPUT

Namespace(path_to_predictions='preds.txt', round=False)
####################
RMSE: 2.494566718354876
####################


## API

接下来我们来构建一个推荐系统的API，第一步是把用户信息取过来转成一个json的请求。

In [5]:
titles = pd.read_csv(MOVIE_TITLES, names=['MovieID','Year','Title'])
titles.head()

,MovieID,Year,Title
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW


In [6]:
target = df[df['CustomerID'] == 84]
target

,CustomerID,MovieID,Rating
10,84,58,3.0
11,84,837,4.0
12,84,319,3.0
13,84,783,3.0
14,84,1011,3.0
15,84,48,3.0


In [7]:
df_customer = pd.merge(target, titles, on='MovieID', how='left')
df_customer.drop(['Title','Year'], axis=1, inplace=True)
df_customer

,CustomerID,MovieID,Rating
0,84,58,3.0
1,84,837,4.0
2,84,319,3.0
3,84,783,3.0
4,84,1011,3.0
5,84,48,3.0


用户的json信息如下，比如这里是对于`CustomerID`为0的用户，有包含`MovieID`和打分的字典。

In [8]:
df_query = df_customer.drop(['CustomerID'], axis=1).set_index('MovieID')
dict_query = df_query.to_dict()['Rating']
dict_query

{'1011': 3.0, '319': 3.0, '48': 3.0, '58': 3.0, '783': 3.0, '837': 4.0}

API定义在[api.py](api.py)文件中，当服务启动的时候，会加载训练好的模型到内存中。main函数 `/recommend`以json形式接收`dict_query`，通过autoencoder完成计算，返回另外一个json(预测打分)

#### <font color=red>在当前目录下打开命令行窗口，输入如下命令启动服务:</font>
python api.py

In [9]:
end_point = 'http://127.0.0.1:5000/'
end_point_recommend = "http://127.0.0.1:5000/recommend"

In [10]:
!curl $end_point

Yeah, yeah, I highly recommend it


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    33  100    33    0     0     33      0  0:00:01 --:--:--  0:00:01  2062


我们通过`dict_query`来向推荐系统发起请求

In [11]:
headers = {'Content-type':'application/json'}
res = requests.post(end_point_recommend, data=json.dumps(dict_query), headers=headers)
print(res.ok)
print(json.dumps(res.json(), indent=2))

False
{
  "error": "Server Internal Error"
}


<font color=red>原始课件中 reco_encoder/data/ 下缺少 input_layer_api.py 这个模块，代码调试时直接用 input_layer.py 这个模块做了修改替换，接口不对应，所以会有错误。</font>

## 压测
如果我们想看一看我们推荐系统的稳定性，可以做一个压测，频繁发起一些请求。大家可以在[load_test.py](load_test.py)中找到压测的代码，我们可以通过调整`NUM`和`CONCURRENT`来控制轮次和并发数。这里测试的服务器`NUM`设定为10，并发为2，最后测试结果大概是4ms左右的响应时间，实际上并发为2的相应速度基本都差不多，如果把并发数提到20的话，响应时间就会上升为12ms

在实际应用中，请求并不从同一台电脑发出，所以大家还得加上客户端和服务器之间的通信延迟等等。

In [12]:
NUM = 10
CONCURRENT = 2
VERBOSE = True
payload = {13:5.0, 191:5.0, 209:5.0}
payload_list = [payload]*NUM

In [13]:
async def main():
    async with aiohttp.ClientSession() as session:
        run_load_test(end_point_recommend, payload_list, session, CONCURRENT, VERBOSE)
        
loop = asyncio.get_event_loop()
loop.run_until_complete(main())